# Text file parsing for iRNA Screening

__Setup__

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import shutil

## iRNA Screening

<img src="./_img/20180803_143938.jpg" width='75%'>

<img src="./_img/image002.png">

## Task

The method of siRNA screening focuses on _switching off_ particular genes in a cell body.
The template comes in a _384 well format_. A robitic instrument with 96 needles does the measurments, hence, during measurment the result is transformed to a 4 x 96 well format. Each of those measurements is then stored in a `.txt` file.

The file names look like this:

```
20180611_112154 NOREAD_PLATE.TXT
20180611_113634 NOREAD_PLATE.TXT
20180611_115108 NOREAD_PLATE.TXT
20180611_120542 NOREAD_PLATE.TXT
20180611_122017 NOREAD_PLATE.TXT
…
```
> __1. The first task is to rename the filnames each corresponding to a block of for files e.g.  `Drugs_A1.1.txt`, `Drugs_A1.2.txt`, ..., `Drugs_A2.1.txt`, ... Note that the files encode the date and time in their filenames.__

The example above would result in something like that:
```
20180611_112154 NOREAD_PLATE.TXT —> Drugs_A1.1.txt
20180611_113634 NOREAD_PLATE.TXT —> Drugs_A1.2.txt
20180611_115108 NOREAD_PLATE.TXT —> Drugs_A1.3.txt
20180611_120542 NOREAD_PLATE.TXT —> Drugs_A1.4.txt
```

> __2. The second task is to extract a particular section of the textifles and store this section in a `.xlsx` file.__

```
Drugs_A1.1.txt —> Drugs_A1.1.xlsx
```



 
 

## Example data

The data is strored in `../data/iRNA_Screening/`

Let us take a loot at one of them:

In [ ]:
! cat ../data/iRNA_Screening/20180611_112154\ NOREAD_PLATE.TXT

## Task 1: Sorting files based on date and time

### Getting the paths

In [ ]:
path =  Path('../data/iRNA_Screening/')

In [ ]:
path.glob('**/20*.TXT')

In [ ]:
list(path.glob('**/20*.TXT'))

In [ ]:
f = list(path.glob('**/20*.TXT'))[0]

In [ ]:
f

### Getting the date and time from the filename

In [ ]:
raw_datetime = f.stem.split(' ')[0]
raw_datetime

In [ ]:
datetime = pd.to_datetime(raw_datetime, format='%Y%m%d_%H%M%S')
datetime

## Apply logic to all files

In [ ]:
out = {}
for f in path.glob('**/20*.TXT'):
    raw_datetime = f.stem.split(' ')[0]
    datetime = pd.to_datetime(raw_datetime, format='%Y%m%d_%H%M%S')
    out[datetime] = f

In [ ]:
out

### Group files based on datetime and rename

In [ ]:
for k in sorted(out.keys()):
    print(k)

In [ ]:
index = list(range(1,int(np.ceil((len(out.keys())+1)/4))))
index

In [ ]:
# set starting values
index_pos = -1 # -1 instead of 0 as 0 % 4 == 0!
sub_index = 1
txt = 'Drugs_A'

for e, k in enumerate(sorted(out.keys())):
    if e % 4 == 0:
        # if e is 0, 4, 8, 12, ... do:
        index_pos += 1
        sub_index = 1
        print (f'\nIndex position = {index_pos}')
    
    # generate new filename
    new_name = f'{txt}{index[index_pos]}.{sub_index}.TXT'
    print(f'Old name: {k} --> New name: {new_name}')
    sub_index += 1
    
    # copy file with new name
    source_name = out[k]
    shutil.copyfile(source_name, path.joinpath(new_name))
    

## Task 2: Extract a particular section of the textifle and store this section in a `.xlsx` file.

In [ ]:
#! cat ../data/iRNA_Screening/Drugs_A1.1.TXT

In [ ]:
#pd.read_csv('../data/iRNA_Screening/Drugs_A1.1.TXT')

### Load data using pandas powerful `read_csv` function

In [ ]:
df_raw = pd.read_csv('../data/iRNA_Screening/Drugs_A1.1.TXT', 
                     skiprows=55, 
                     skipfooter=79, 
                     sep='\t', 
                     encoding='utf-16', 
                     engine='python')
df_raw

### Subset the data

In [ ]:
df = df_raw[df_raw.columns[2:14]]
df.columns = range(1,13)
df

### Apply logic to all files and safe them as `.xlsx` files

In [ ]:
list(path.glob('**/Drug*.TXT'))

In [ ]:
for f in path.glob('**/Drug*.TXT'):
    # load the data
    df_raw = pd.read_csv(f, 
                         skiprows=55, 
                         skipfooter=79, 
                         sep='\t', 
                         encoding='utf-16', 
                         engine='python')
    # subset
    df = df_raw[df_raw.columns[2:14]]
    df.columns = range(1,13)
    # save
    filename = Path(*f.parts[:-1]).joinpath(f.parts[-1].replace('TXT', 'xlsx'))
    df.to_excel(filename, index=False)
    

***